In [8]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.3 MB/s eta 0:00:00
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 12.5 MB/s eta 0:00:00
  Using cached soupsieve-2.4.1-py3-none-any.whl (36 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [12]:
from Bio import Entrez
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [13]:
genes_of_interest = [
    'ABCC8', 'ACTB', 'ACTG1', 'ADAR', 'ADCY5', 'ADNP',
    'AFG3L2', 'AHDC1', 'AIFM1', 'ALDH3A2', 'AMPD2', 'AP1S2',
    'AP4M1', 'AP4S1', 'ARID1B', 'ARID2', 'ARSE', 'ARX',
    'ASXL1', 'ASXL3', 'ATL1', 'ATM', 'ATP1A2', 'ATP1A3',
    'ATP5A1', 'ATP6V0A4', 'ATP7A', 'ATRX', 'AUTS2', 'BCAP31',
    'BCL11A', 'C5orf42', 'CACNA1A', 'CACNA1G', 'CALM1',
    'CAMTA1', 'CASK', 'CDC42', 'CDKL5', 'CHD2', 'CHD8',
    'CHMP1A', 'CHRNE', 'CIT', 'CLTC', 'CLTCL1', 'COL4A1',
    'COL4A2', 'CREBBP', 'CTBP1', 'CTC1', 'CTNNB1', 'CUL4B',
    'CYP2U1', 'DDC', 'DDHD2', 'DDX3X', 'DKC1', 'DNM1',
    'DNM1L', 'DNMT3A', 'DOCK6', 'DYNC1H1', 'DYRK1A',
    'EARS2', 'EBF3', 'ECHS1', 'EEF1A2', 'EHMT1', 'EIF2B2',
    'EP300', 'EPG5', 'ERCC6', 'ERCC8', 'EXOSC3', 'EZH2',
    'FA2H', 'FAR1', 'FARS2', 'FBN2', 'FGD1', 'FGF12', 'FGFR1',
    'FH', 'FOXG1', 'FRRS1L', 'FUCA1', 'G6PD', 'GABRA1',
    'GABRB2', 'GFAP', 'GLRA1', 'GNAO1', 'GNAS', 'GNB1',
    'GPT2', 'GRIA3', 'GRIN1', 'GRIN2B', 'HACE1', 'HDAC8',
    'HECW2', 'HIST1H1E', 'HNRNPH2', 'HPRT1', 'IDS', 'IFIH1',
    'IQSEC2', 'ITPR1', 'KAT6A', 'KCNB1', 'KCNC3', 'KCNQ2',
    'KCNT1', 'KCTD7', 'KDM5C', 'KIAA2022/NEXMIF', 'KIDINS220',
    'KIF11', 'KIF1A', 'KMT2A', 'KMT2B', 'L1CAM', 'LIPH',
    'LMAN2L', 'MAP2K1', 'MBD5', 'MECP2', 'MED12', 'MED13L',
    'MEF2C', 'MFN2', 'MICU1', 'MOCS2', 'MTFMT', 'NAA10',
    'NACC1', 'NALCN', 'NGLY1', 'NIPBL', 'NKX2-1', 'NKX6-2',
    'OPA1', 'OPHN1', 'OTUD6B', 'PANK2', 'PCDH12', 'PDHA1',
    'PDHX', 'PEX16', 'PGAP1', 'PHIP', 'PHKA1', 'PIEZO2',
    'PIGA', 'PIGN', 'PIGV', 'PIK3CD', 'PLA2G6', 'PLOD1',
    'PLP1', 'PNKP', 'POGZ', 'POLG1', 'POMGNT1', 'PPM1D',
    'PPT1', 'PRUNE1', 'PTPN11', 'PURA', 'RAB3GAP1', 'RAB3GAP2',
    'REEP1', 'RNASEH2B', 'RPE65', 'RYR1', 'RYR2', 'SACS',
    'SATB2', 'SCN1A', 'SCN2A', 'SCN8A', 'SEPSECS', 'SETBP1',
    'SETD2', 'SETD5', 'SHANK3', 'SHOC2', 'SIL1', 'SIN3A',
    'SLC13A5', 'SLC16A2', 'SLC1A2', 'SLC1A4', 'SLC2A1',
    'SLC6A3', 'SMARCA2', 'SMARCB1', 'SNAP29', 'SON', 'SOX2',
    'SPAST', 'SPATA5', 'SPG11', 'SPR', 'SPTAN1', 'STAMBP',
    'STAT3', 'STXBP1', 'SUMF1', 'SYNE1', 'SYNGAP1', 'TAF1',
    'TANGO2', 'TBCK', 'TBL1XR1', 'TCF4', 'TELO2', 'TOE1',
    'TPP1', 'TRAPPC9', 'TREX1', 'TRIP12', 'TSEN54', 'TUBA1A',
    'TUBB2A', 'TUBB2B', 'TUBB3', 'TUBB4A', 'UBA5', 'UBE3A',
    'UPF3B', 'USP9X', 'VPS11', 'WAC', 'WDR26', 'WDR45', 'WWOX',
    'YY1', 'ZC4H2', 'ZDHHC9', 'ZEB2', 'ZNF423'
]

In [5]:
Entrez.email = "tylerkruer@arizona.edu"

def create_search_terms(gene_or_phenotype):
    base_search = f"{gene_or_phenotype} [treatment or intervention]"
    filter = "AND patient"
    return base_search + f"{filter}", filter

def search_and_save_to_csv(db, filename):
    results = []
    
    for gene in genes_of_interest:
        print(f"Processing gene: {gene} for database: {db}")
        search_term, filter_used = create_search_terms(gene)

        handle = Entrez.esearch(db=db, term=search_term, retmax=10000)
        record = Entrez.read(handle)
        handle.close()

        num_results = len(record["IdList"])
        print(f"Found {num_results} results for {gene} with search term: {search_term}")
        results.append((gene, search_term, filter_used, num_results))

    with open(filename, "w", newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Gene", "Search Term", "Search Filter", "Number of Results"])
        
        for gene, search_term, filter_used, num_results in results:
            writer.writerow([gene, search_term, filter_used, num_results])
    
    print(f"Data written to {filename}")

search_and_save_to_csv("pubmed", "../output/pubmed_output.csv")

Processing gene: ABCC8 for database: pubmed
Found 578 results for ABCC8 with search term: ABCC8 [treatment or intervention]AND patient
Processing gene: ACTB for database: pubmed
Found 320 results for ACTB with search term: ACTB [treatment or intervention]AND patient
Processing gene: ACTG1 for database: pubmed
Found 86 results for ACTG1 with search term: ACTG1 [treatment or intervention]AND patient
Processing gene: ADAR for database: pubmed
Found 414 results for ADAR with search term: ADAR [treatment or intervention]AND patient
Processing gene: ADCY5 for database: pubmed
Found 63 results for ADCY5 with search term: ADCY5 [treatment or intervention]AND patient
Processing gene: ADNP for database: pubmed
Found 69 results for ADNP with search term: ADNP [treatment or intervention]AND patient
Processing gene: AFG3L2 for database: pubmed
Found 50 results for AFG3L2 with search term: AFG3L2 [treatment or intervention]AND patient
Processing gene: AHDC1 for database: pubmed
Found 29 results for 

In [14]:
with open('../output/ncbi_books_output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Gene', 'Result']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)    
    
    writer.writeheader()

    for gene in genes_of_interest:
        print(f"Processing gene: {gene} from NCBI Books")
        time.sleep(1)
        url = f"https://www.ncbi.nlm.nih.gov/books/?term={gene}"

        page = requests.get(url)

        soup = BeautifulSoup(page.content, 'html.parser')

        element = soup.select_one("#maincontent > div > div:nth-child(3) > div > h3")
        
        if element:
            print(element.text)
            

            writer.writerow({'Gene': gene, 'Result': element.text})
            
        else:
            print("Element not found.")
            

            writer.writerow({'Gene': gene, 'Result': 'Element not found'})

print("Web scraping completed and data saved to ncbi_books_output.csv.")


Processing gene: ABCC8 from NCBI Books
Results: 8 books (122 items)
Processing gene: ACTB from NCBI Books
Results: 12 books (57 items)
Processing gene: ACTG1 from NCBI Books
Results: 3 books (61 items)
Processing gene: ADAR from NCBI Books
Results: 1 to 20 of 42 books (141 items)
Processing gene: ADCY5 from NCBI Books
Results: 3 books (45 items)
Processing gene: ADNP from NCBI Books
Results: 5 books (30 items)
Processing gene: AFG3L2 from NCBI Books
Results: 2 books (48 items)
Processing gene: AHDC1 from NCBI Books
Results: 2 books (21 items)
Processing gene: AIFM1 from NCBI Books
Results: 2 books (37 items)
Processing gene: ALDH3A2 from NCBI Books
Results: 2 books (30 items)
Processing gene: AMPD2 from NCBI Books
Element not found.
Processing gene: AP1S2 from NCBI Books
Element not found.
Processing gene: AP4M1 from NCBI Books
Results: 2 books (50 items)
Processing gene: AP4S1 from NCBI Books
Results: 2 books (50 items)
Processing gene: ARID1B from NCBI Books
Results: 7 books (87 item